In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification

In [2]:
np.logspace(-4, 4, 10)

array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])

### Generate Dataset

In [3]:
X, y = make_classification(
    n_features=10, 
    n_samples=1000, 
    n_informative=8,
    n_redundant=2,
    n_repeated=0,
    n_classes=2, 
    random_state=42
)

### Hyperparameter Tunning Using GridSearchCV

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = LogisticRegression(max_iter=2000)

# Define the parameter grid
param_grid = {
    'C': [1, 2, 3, 4, 5],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']  # Different solvers
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, scoring='accuracy')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test set evaluation:")
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Best parameters found:  {'C': 1, 'solver': 'liblinear'}
Best cross-validation score: 0.72
Test set evaluation:
              precision    recall  f1-score   support

           0       0.73      0.62      0.67       106
           1       0.63      0.73      0.68        94

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.68      0.68      0.67       200



In [5]:
%pip install optuna


  Using cached optuna-4.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.16.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.3.0-py3-none-any.whl (386 kB)
Using cached alembic-1.16.1-py3-none-any.whl (242 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Hyperparameter Tunning Using Optuna

In [6]:
import optuna
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [7]:
def objective(trial):
    C = trial.suggest_float("C", 1,5)
    solver = trial.suggest_categorical("solver",["newton-cg", "lbfgs", "liblinear", "sag", "saga"] )
    model = LogisticRegression(C=C, solver=solver, max_iter=1000)
    scores = cross_val_score(model, X, y, cv=3, scoring='accuracy')
    accuracy = np.mean(scores)
    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best Params: ", study.best_params)
print("Best cv acc: ", study.best_value)

[I 2025-06-06 10:57:49,192] A new study created in memory with name: no-name-808f3483-f73c-47e6-a7cb-57bd925b8413
[I 2025-06-06 10:57:49,202] Trial 0 finished with value: 0.687996379613146 and parameters: {'C': 1.5437450249428895, 'solver': 'liblinear'}. Best is trial 0 with value: 0.687996379613146.
[I 2025-06-06 10:57:49,217] Trial 1 finished with value: 0.687996379613146 and parameters: {'C': 2.4616229373994143, 'solver': 'sag'}. Best is trial 0 with value: 0.687996379613146.
[I 2025-06-06 10:57:49,228] Trial 2 finished with value: 0.687996379613146 and parameters: {'C': 2.3549224676235694, 'solver': 'saga'}. Best is trial 0 with value: 0.687996379613146.
[I 2025-06-06 10:57:49,242] Trial 3 finished with value: 0.687996379613146 and parameters: {'C': 4.997128680070544, 'solver': 'sag'}. Best is trial 0 with value: 0.687996379613146.
[I 2025-06-06 10:57:49,257] Trial 4 finished with value: 0.687996379613146 and parameters: {'C': 4.644590543421948, 'solver': 'sag'}. Best is trial 0 wi

Best Params:  {'C': 1.5437450249428895, 'solver': 'liblinear'}
Best cv acc:  0.687996379613146
